In [1]:
import json
import os

import tqdm

import src.visualization.fit_images as fi

In [2]:
required_paths = ["/ai4eutils", "/CameraTraps", "/yolov5"]
python_path = os.environ.get("PYTHONPATH", "")
root_path = os.getcwd()

for path in required_paths:
    if not any(p.endswith(path) for p in python_path.split(":")):
        python_path += f":{root_path}/data/external{path}"

os.environ["PYTHONPATH"] = python_path

!echo "PYTHONPATH: $PYTHONPATH"

PYTHONPATH: :/Users/carlos/WORKSPACE/MegaClassifier/data/external/ai4eutils:/Users/carlos/WORKSPACE/MegaClassifier/data/external/CameraTraps:/Users/carlos/WORKSPACE/MegaClassifier/data/external/yolov5


In [3]:
DATASET_PATH = os.path.abspath("./data/raw/emptyNonEmptyDataset_ETL")
IMAGE_PATH = os.path.abspath("./data/interim/masked_images")
MASK_PATH = os.path.abspath("./resources/mask")
FIT_PATH = os.path.abspath("./data/interim/fit_images")

DETECTION_JSON = os.path.abspath("./resources/json/28560_images_0_0015_threshold.json")

with open(DETECTION_JSON, "r") as f:
    result = json.load(f)

IMAGE_PATH = os.path.join(IMAGE_PATH, os.path.basename(DATASET_PATH))

MASK_PATH = os.path.join(MASK_PATH, os.path.basename(DATASET_PATH))

FIT_PATH = os.path.join(
    FIT_PATH,
    os.path.basename(DATASET_PATH),
)


print()
print(f"DATASET_PATH:   {DATASET_PATH}")
print(f"IMAGE_PATH:     {IMAGE_PATH}")
print(f"MASK_PATH:      {MASK_PATH}")
print(f"FIT_PATH:       {FIT_PATH}")
print(f"DETECTION_JSON: {DETECTION_JSON}")


DATASET_PATH:   /Users/carlos/WORKSPACE/MegaClassifier/data/raw/emptyNonEmptyDataset_ETL
IMAGE_PATH:     /Users/carlos/WORKSPACE/MegaClassifier/data/interim/masked_images/emptyNonEmptyDataset_ETL
MASK_PATH:      /Users/carlos/WORKSPACE/MegaClassifier/resources/mask/emptyNonEmptyDataset_ETL
FIT_PATH:       /Users/carlos/WORKSPACE/MegaClassifier/data/interim/fit_images/emptyNonEmptyDataset_ETL
DETECTION_JSON: /Users/carlos/WORKSPACE/MegaClassifier/resources/json/28560_images_0_0015_threshold.json


In [4]:
for image in tqdm.tqdm(result["images"]):
    image_file = os.path.join(IMAGE_PATH, image["file"])
    mask_file = os.path.join(MASK_PATH, image["file"])
    fit_file = os.path.join(FIT_PATH, image["file"])

    fi.fit_image(image_file, mask_file, fit_file)


100%|██████████| 28560/28560 [04:53<00:00, 97.29it/s] 
